In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/silver/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2c2a6cc3-01aa-4add-9096-7425242b62ac;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 14519ms :: artifacts dl 31ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-r

In [ ]:
# apagar tudo do hdfs

#to_path = "/projeto/fisico/bronze/speed_distance/"
#to_path_bronze = "/projeto/fisico/bronze/speed_distance/"

#client.delete(to_path,True)
#client.delete(to_path_silver,True)

In [2]:
# declaracao dos caminhos

from_path = "../projeto/bronze/speed_distance/"

hdfs_path_2021 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2020-21.csv"
hdfs_path_1920 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2019-20.csv"
hdfs_path_1819 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2018-19.csv"
hdfs_path_1718 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2017-18.csv"
hdfs_path_1617 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2016-17.csv"
hdfs_path_1516 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2015-16.csv"
hdfs_path_1415 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2014-15.csv"
hdfs_path_1314 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/speed_distance/2013-14.csv"

In [ ]:
# upload de bronze

client.upload(to_path, from_path)

In [3]:
df2021 = spark.read.csv(hdfs_path_2021, header= True)
df1920 = spark.read.csv(hdfs_path_1920, header= True)
df1819 = spark.read.csv(hdfs_path_1819, header= True)
df1718 = spark.read.csv(hdfs_path_1718, header= True)
df1617 = spark.read.csv(hdfs_path_1617, header= True)
df1516 = spark.read.csv(hdfs_path_1516, header= True)
df1415 = spark.read.csv(hdfs_path_1415, header= True)
df1314 = spark.read.csv(hdfs_path_1314, header= True)

In [ ]:
# alterar conforme necessário

df2021.printSchema()
#df2021.show(truncate=False)
#df2021.toPandas()

In [ ]:
#df2021_status = df2021.summary() #sempre que se cria uma nova funçao é necessario um novo dataframe
#df1920_status = df1920.summary() #sempre que se cria uma nova funçao é necessario um novo dataframe

#df2021_status.toPandas() #serve para msotrar a informação de cima obtida
#df1920_status.toPandas() #serve para msotrar a informação de cima obtida

In [4]:
# INICIO DA JUNÇÃO

In [5]:
# adicionar colunas da season

df2021_f = df2021.withColumn(
    "SEASON",
    lit("2020/2021")
    )
df1920_f = df1920.withColumn(
    "SEASON",
    lit("2019/2020")
    )
df1819_f = df1819.withColumn(
    "SEASON",
    lit("2018/2019")
    )
df1718_f = df1718.withColumn(
    "SEASON",
    lit("2017/2018")
    )
df1617_f = df1617.withColumn(
    "SEASON",
    lit("2016/2017")
    )
df1516_f = df1516.withColumn(
    "SEASON",
    lit("2015/2016")
    )
df1415_f = df1415.withColumn(
    "SEASON",
    lit("2014/2015")
    )
df1314_f = df1314.withColumn(
    "SEASON",
    lit("2013/2014")
    )

In [6]:
# uniao dos dataframes todos

full_df = df2021_f.union(df1920_f.union(df1718_f.union(df1617_f.union(df1617_f.union(df1516_f.union(df1415_f.union(df1314_f)))))))

In [7]:
full_df.printSchema()
full_df.show(truncate=False)
full_df.toPandas()

root
 |-- Player: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- W: string (nullable = true)
 |-- L: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- DIST. FEET: string (nullable = true)
 |-- DIST. MILES: string (nullable = true)
 |-- DIST. MILES OFF: string (nullable = true)
 |-- DIST. MILES DEF: string (nullable = true)
 |-- AVG SPEED: string (nullable = true)
 |-- AVG SPEED OFF: string (nullable = true)
 |-- AVG SPEED DEF: string (nullable = true)
 |-- SEASON: string (nullable = false)

+------------------+----+---+---+---+----+----------+-----------+---------------+---------------+---------+-------------+-------------+---------+
|Player            |TEAM|GP |W  |L  |MIN |DIST. FEET|DIST. MILES|DIST. MILES OFF|DIST. MILES DEF|AVG SPEED|AVG SPEED OFF|AVG SPEED DEF|SEASON   |
+------------------+----+---+---+---+----+----------+-----------+---------------+---------------+---------+-------------+-------------+---------

,Player,TEAM,GP,W,L,MIN,DIST. FEET,DIST. MILES,DIST. MILES OFF,DIST. MILES DEF,AVG SPEED,AVG SPEED OFF,AVG SPEED DEF,SEASON
0,Aaron Gordon,DEN,50,29,21,1384,523828,99.20,54.90,44.40,4.06,4.40,3.71,2020/2021
1,Aaron Holiday,IND,66,30,36,1176,479903,90.90,47.00,43.90,4.32,4.70,3.98,2020/2021
2,Aaron Nesmith,BOS,46,22,24,669,274976,52.10,27.20,24.90,4.34,4.53,4.15,2020/2021
3,Abdel Nader,PHX,24,16,8,355,144913,27.40,14.80,12.70,4.41,4.75,4.10,2020/2021
4,Adam Mokoka,CHI,14,3,11,56,22367,4.20,2.20,2.10,4.13,4.48,3.99,2020/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,Willie Green,LAC,55,38,17,869,329326,62.40,32.30,30.10,4.32,4.58,4.06,2013/2014
4027,Wilson Chandler,DEN,62,27,35,1927,717124,135.80,71.20,64.60,4.24,4.62,3.90,2013/2014
4028,Xavier Henry,LAL,43,16,27,908,357114,67.60,35.00,32.70,4.49,4.82,4.20,2013/2014
4029,Zach Randolph,MEM,79,49,30,2705,922134,174.60,92.20,82.40,3.88,3.90,3.85,2013/2014


In [8]:
# INÍCIO DA EDIÇÃO DO DATAFRAME

In [9]:
replaced_df = full_df.withColumnRenamed("DIST. FEET","DIST_FEET") \
    .withColumnRenamed("DIST. MILES","DIST_MILES") \
    .withColumnRenamed("DIST. MILES OFF","DIST_MILES_OFF") \
    .withColumnRenamed("DIST. MILES DEF","DIST_MILES_DEF")

In [10]:
replaced_df = replaced_df.withColumnRenamed("AVG SPEED","AVG_SPEED_MPH") \
    .withColumnRenamed("AVG SPEED OFF","AVG_SPEED_MPH_OFF") \
    .withColumnRenamed("AVG SPEED DEF","AVG_SPEED_MPH_DEF") \
    .withColumnRenamed("Player","PLAYER")

In [11]:
replaced_df = replaced_df.withColumn(
    "DIST_KM",
    col("DIST_FEET") *  0.0003048
    ) \
    .withColumn(    
    "DIST_KM_OFF",
    col("DIST_MILES_OFF") * 1.609344
    ) \
    .withColumn(    
    "DIST_KM_DEF",
    col("DIST_MILES_DEF") * 1.609344
    )

In [12]:
replaced_df = replaced_df.withColumn(
    "AVG_SPEED_KMH",
    col("AVG_SPEED_MPH") *  1.609344
    ) \
    .withColumn(    
    "AVG_SPEED_KMH_OFF",
    col("AVG_SPEED_MPH_OFF") * 1.609344
    ) \
    .withColumn(    
    "AVG_SPEED_KMH_DEF",
    col("AVG_SPEED_MPH_DEF") * 1.609344
    )

In [13]:
replaced_df = replaced_df.withColumn(
    "DIST_KM_PER_MIN",
    col("DIST_KM") / col("MIN")
    ) \
    .withColumn(    
    "DIST_KM_OFF_PER_MIN",
    col("DIST_KM_OFF") / col("MIN")
    ) \
    .withColumn(    
    "DIST_KM_DEF_PER_MIN",
    col("DIST_KM_DEF") / col("MIN")
    )

In [14]:
replaced_df = replaced_df.withColumn(
    "AVG_SPEED_KMH_PER_MIN",
    col("AVG_SPEED_KMH") / col("MIN")
    ) \
    .withColumn(    
    "AVG_SPEED_KMH_OFF_PER_MIN",
    col("AVG_SPEED_KMH_OFF") / col("MIN")
    ) \
    .withColumn(    
    "AVG_SPEED_KMH_DEF_PER_MIN",
    col("AVG_SPEED_KMH_DEF") / col("MIN")
    )

In [15]:
replaced_df = replaced_df.withColumn(
    "AVG_MIN_PER_GAME",
    col("MIN") / col("GP")
    )

In [16]:
# FIM DA EDIÇÃO DO DATAFRAME

In [17]:
replaced_df = replaced_df.withColumn("PLAYER", replaced_df["PLAYER"].cast(StringType())) 
replaced_df = replaced_df.withColumn("TEAM", replaced_df["TEAM"].cast(StringType()))
replaced_df = replaced_df.withColumn("GP", replaced_df["GP"].cast(IntegerType())) 
replaced_df = replaced_df.withColumn("W", replaced_df["W"].cast(IntegerType()))
replaced_df = replaced_df.withColumn("L", replaced_df["L"].cast(IntegerType())) 
replaced_df = replaced_df.withColumn("MIN", replaced_df["MIN"].cast(IntegerType())) 
replaced_df = replaced_df.withColumn("DIST_FEET",replaced_df["DIST_FEET"].cast(DoubleType())) 
replaced_df = replaced_df.withColumn("DIST_MILES", replaced_df["DIST_MILES"].cast(DoubleType())) 
replaced_df = replaced_df.withColumn("DIST_MILES_OFF", replaced_df["DIST_MILES_OFF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_MILES_DEF", replaced_df["DIST_MILES_DEF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_MPH", replaced_df["AVG_SPEED_MPH"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_MPH_OFF", replaced_df["AVG_SPEED_MPH_OFF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_MPH_DEF", replaced_df["AVG_SPEED_MPH_DEF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("SEASON", replaced_df["SEASON"].cast(StringType()))
replaced_df = replaced_df.withColumn("DIST_KM", replaced_df["DIST_KM"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_KM_OFF", replaced_df["DIST_KM_OFF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_KM_DEF", replaced_df["DIST_KM_DEF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_KMH", replaced_df["AVG_SPEED_KMH"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_KMH_OFF", replaced_df["AVG_SPEED_KMH_OFF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_KMH_DEF", replaced_df["AVG_SPEED_KMH_DEF"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_KM_PER_MIN", replaced_df["DIST_KM_PER_MIN"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_KM_OFF_PER_MIN", replaced_df["DIST_KM_OFF_PER_MIN"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("DIST_KM_DEF_PER_MIN", replaced_df["DIST_KM_DEF_PER_MIN"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("AVG_SPEED_KMH_PER_MIN", replaced_df["AVG_SPEED_KMH_PER_MIN"].cast(DoubleType()))

In [18]:
replaced_df.printSchema()
replaced_df.show(truncate=False)
replaced_df.toPandas()
#topandas usar apenas para mostrar os dados em tabela

root
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- DIST_FEET: double (nullable = true)
 |-- DIST_MILES: double (nullable = true)
 |-- DIST_MILES_OFF: double (nullable = true)
 |-- DIST_MILES_DEF: double (nullable = true)
 |-- AVG_SPEED_MPH: double (nullable = true)
 |-- AVG_SPEED_MPH_OFF: double (nullable = true)
 |-- AVG_SPEED_MPH_DEF: double (nullable = true)
 |-- SEASON: string (nullable = false)
 |-- DIST_KM: double (nullable = true)
 |-- DIST_KM_OFF: double (nullable = true)
 |-- DIST_KM_DEF: double (nullable = true)
 |-- AVG_SPEED_KMH: double (nullable = true)
 |-- AVG_SPEED_KMH_OFF: double (nullable = true)
 |-- AVG_SPEED_KMH_DEF: double (nullable = true)
 |-- DIST_KM_PER_MIN: double (nullable = true)
 |-- DIST_KM_OFF_PER_MIN: double (nullable = true)
 |-- DIST_KM_DEF_PER_MIN: double (nullable = true)
 |-- AVG

22/01/13 20:22:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+----+---+---+---+----+---------+----------+--------------+--------------+-------------+-----------------+-----------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+--------------------+---------------------+-------------------------+-------------------------+------------------+
|PLAYER            |TEAM|GP |W  |L  |MIN |DIST_FEET|DIST_MILES|DIST_MILES_OFF|DIST_MILES_DEF|AVG_SPEED_MPH|AVG_SPEED_MPH_OFF|AVG_SPEED_MPH_DEF|SEASON   |DIST_KM           |DIST_KM_OFF       |DIST_KM_DEF       |AVG_SPEED_KMH     |AVG_SPEED_KMH_OFF |AVG_SPEED_KMH_DEF |DIST_KM_PER_MIN    |DIST_KM_OFF_PER_MIN |DIST_KM_DEF_PER_MIN |AVG_SPEED_KMH_PER_MIN|AVG_SPEED_KMH_OFF_PER_MIN|AVG_SPEED_KMH_DEF_PER_MIN|AVG_MIN_PER_GAME  |
+------------------+----+---+---+---+----+---------+----------+--------------+--------------+-------------+-----------------+-----------------+---------+-

,PLAYER,TEAM,GP,W,L,MIN,DIST_FEET,DIST_MILES,DIST_MILES_OFF,DIST_MILES_DEF,...,AVG_SPEED_KMH,AVG_SPEED_KMH_OFF,AVG_SPEED_KMH_DEF,DIST_KM_PER_MIN,DIST_KM_OFF_PER_MIN,DIST_KM_DEF_PER_MIN,AVG_SPEED_KMH_PER_MIN,AVG_SPEED_KMH_OFF_PER_MIN,AVG_SPEED_KMH_DEF_PER_MIN,AVG_MIN_PER_GAME
0,Aaron Gordon,DEN,50,29,21,1384,523828.0,99.2,54.9,44.4,...,6.533937,7.081114,5.970666,0.115363,0.063839,0.051629,0.004721,0.005116,0.004314,27.680000
1,Aaron Holiday,IND,66,30,36,1176,479903.0,90.9,47.0,43.9,...,6.952366,7.563917,6.405189,0.124383,0.064319,0.060077,0.005912,0.006432,0.005447,17.818182
2,Aaron Nesmith,BOS,46,22,24,669,274976.0,52.1,27.2,24.9,...,6.984553,7.290328,6.678778,0.125281,0.065432,0.059899,0.010440,0.010897,0.009983,14.543478
3,Abdel Nader,PHX,24,16,8,355,144913.0,27.4,14.8,12.7,...,7.097207,7.644384,6.598310,0.124421,0.067094,0.057574,0.019992,0.021533,0.018587,14.791667
4,Adam Mokoka,CHI,14,3,11,56,22367.0,4.2,2.2,2.1,...,6.646591,7.209861,6.421283,0.121740,0.063224,0.060350,0.118689,0.128748,0.114666,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,Willie Green,LAC,55,38,17,869,329326.0,62.4,32.3,30.1,...,6.952366,7.370796,6.533937,0.115510,0.059818,0.055744,0.008000,0.008482,0.007519,15.800000
4027,Wilson Chandler,DEN,62,27,35,1927,717124.0,135.8,71.2,64.6,...,6.823619,7.435169,6.276442,0.113430,0.059463,0.053951,0.003541,0.003858,0.003257,31.080645
4028,Xavier Henry,LAL,43,16,27,908,357114.0,67.6,35.0,32.7,...,7.225955,7.757038,6.759245,0.119877,0.062034,0.057958,0.007958,0.008543,0.007444,21.116279
4029,Zach Randolph,MEM,79,49,30,2705,922134.0,174.6,92.2,82.4,...,6.244255,6.276442,6.195974,0.103906,0.054855,0.049024,0.002308,0.002320,0.002291,34.240506


In [19]:
replaced_df \
    .select("PLAYER","TEAM","SEASON","GP","W","L","MIN","AVG_MIN_PER_GAME","DIST_FEET","DIST_MILES","DIST_MILES_OFF","DIST_MILES_DEF","DIST_KM","DIST_KM_OFF","DIST_KM_DEF","DIST_KM_PER_MIN","DIST_KM_OFF_PER_MIN","DIST_KM_DEF_PER_MIN","AVG_SPEED_MPH","AVG_SPEED_MPH_OFF","AVG_SPEED_MPH_DEF","AVG_SPEED_KMH","AVG_SPEED_KMH_OFF","AVG_SPEED_KMH_DEF","AVG_SPEED_KMH_PER_MIN","AVG_SPEED_KMH_OFF_PER_MIN","AVG_SPEED_KMH_DEF_PER_MIN") \
    .write \
    .mode("overwrite") \
    .partitionBy("SEASON") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/silver/speed_distance")